In [1]:
## Uncomment and install these dependencies
# %pip install -U fastembed
# %pip install -U datasets
# %pip install -U qdrant-client

In [2]:
#fastembed is a fast and lightweight library creating embeddings.
from fastembed import (
    TextEmbedding
)

### Embedding Models

In [3]:
# Unimodal english model
text_model = TextEmbedding(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
)
# Unimodal code model
code_model = TextEmbedding(
    model_name = "jinaai/jina-embeddings-v2-base-code"
)

In [4]:
# sample representation 
text_embeddings = list(text_model.embed(["Hello, world!"*100]*256))
text_embeddings_dim = len(text_embeddings[0])
print("text embedding length: ",text_embeddings_dim)


text embedding length:  384


In [5]:
code_embeddings = list(code_model.embed(["print('Hello, world!')"*100]*10,max_length=1024))
code_embeddings_dim = len(code_embeddings[0])
print("code embedding length: ",code_embeddings_dim)

code embedding length:  768


### Dataset

This dataset contains Problem , sample test cases and thier submissions from Codeforces. 
We will filter out the submissions which have verdict other than "OK" and then concatenate the problem description, input-specification, output-specification, demo-input, and demo-output into one string.


In [4]:
from datasets import load_dataset

ds = load_dataset("MatrixStudio/Codeforces-Python-Submissions",split="train")

Using the latest cached version of the dataset since MatrixStudio/Codeforces-Python-Submissions couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /Users/abcom/.cache/huggingface/datasets/MatrixStudio___Codeforces-Python-Submissions/default/0.0.0/8764ee1b76dcbebed1c6b327c046ebb74219ff95 (last modified on Tue Feb 18 18:16:53 2025).


In [87]:
ds[0]

{'contestId': 604,
 'index': 'A',
 'name': 'Uncowed Forces',
 'type': 'PROGRAMMING',
 'rating': 1000,
 'tags': ['implementation'],
 'title': '',
 'time-limit': None,
 'memory-limit': None,
 'problem-description': "Kevin Sun has just finished competing in Codeforces Round #334! The round was 120 minutes long and featured five problems with maximum point values of 500, 1000, 1500, 2000, and 2500, respectively. Despite the challenging tasks, Kevin was uncowed and bulldozed through all of them, distinguishing himself from the herd as the best cowmputer scientist in all of Bovinia. Kevin knows his submission time for each problem, the number of wrong submissions that he made on each problem, and his total numbers of successful and unsuccessful hacks. Because Codeforces scoring is complicated, Kevin wants you to write a program to compute his final score.\n\nCodeforces scores are computed as follows: If the maximum point value of a problem is *x*, and Kevin submitted correctly at minute *m* 

In [7]:
ds.shape

(621356, 29)

In [9]:
# filter ds where the verdict is "OK"
ds = ds.filter(lambda x: x['verdict'] == "OK")
ds.shape

Filter:   0%|          | 0/621356 [00:00<?, ? examples/s]

(274883, 29)

In [10]:
# Concatenate problem description, input-specification, output-specification, demo-input, and demo-output into one string
problem_description = ds['problem-description']
input_specification = ds['input-specification']
output_specification = ds['output-specification']
demo_input = ds['demo-input']
demo_output = ds['demo-output']

text_data = [f"Problem Description: {desc} \n Input Specification: {inp} \n Output Specification: {out} \n Demo Input: {d_in} \n Demo Output: {d_out}" for desc, inp, out, d_in, d_out in zip(problem_description, input_specification, output_specification, demo_input, demo_output)]

# Concatenate the other code part into another string
code_data = ds['code']


### Collection Creation with Config
We will create two collections, one for the text data and one for the code data.


In [15]:
from qdrant_client import QdrantClient, models

client = QdrantClient("localhost:6333")

client.create_collection(
    collection_name="quantized_collection_for_code_search",
    vectors_config={
        "text": models.VectorParams(size=text_embeddings_dim, distance=models.Distance.COSINE,on_disk=True),
        "code": models.VectorParams(size=code_embeddings_dim, distance=models.Distance.COSINE,on_disk=True)
    },
    quantization_config=models.ScalarQuantization(
        scalar=models.ScalarQuantizationConfig(
            type=models.ScalarType.INT8,
            quantile=0.99,
            always_ram=True,
        ),
    ),
)
client.create_collection(
    collection_name="normal_collection_for_code_search",
    vectors_config={
        "text": models.VectorParams(size=text_embeddings_dim, distance=models.Distance.COSINE,on_disk=True),
        "code": models.VectorParams(size=code_embeddings_dim, distance=models.Distance.COSINE,on_disk=True)
    },
)

/var/folders/_7/yfpb4l0n67x4d3vv8h4g_nd00000gn/T/ipykernel_23884/1260811071.py:5: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(
/var/folders/_7/yfpb4l0n67x4d3vv8h4g_nd00000gn/T/ipykernel_23884/1260811071.py:19: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

Notice how we keep dense vectors on disk and the quantized vectors in memory for saving memory. This approach helps in efficiently managing the memory usage while still allowing fast access to the quantized vectors.


### Upsert Vectors into Qdrant Collection

We create code and text vector and insert it in batches in Qdrant

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

def create_point(text_data, code_data, i):
    try:
        text_vector = list(text_model.embed(text_data))[0]
        code_vector = list(code_model.embed(code_data,max_length=1024))[0]
    except IndexError:
        print(f"Error: Failed to generate embeddings for index {i}")
        return None
    return models.PointStruct(
        id=i,
        vector={"text": text_vector, "code": code_vector},
        payload={"file_content": text_data, "code": code_data},
    )

def create_batch_points(points, batch_size=256):
    """Helper function to create batches of points."""
    for i in range(0, len(points), batch_size):
        yield points[i:i + batch_size]

# Ensure data is non-empty and of the same length
if not text_data or not code_data or len(text_data) != len(code_data):
    raise ValueError("Text data and code data must be non-empty and of the same length.")

# Process embeddings and upsert in batches
batch_size = 256
for start in tqdm(range(0, len(text_data), batch_size), desc="Processing batches"):
    end = start + batch_size
    text_batch = text_data[start:end]
    code_batch = code_data[start:end]

    print("Starting to create points")
    points = []
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = {executor.submit(create_point, text_batch[i], code_batch[i], start + i): i for i in range(len(text_batch))}
        for future in tqdm(as_completed(futures), desc="Creating points", total=len(text_batch)):
            point = future.result()
            if point is not None:
                points.append(point)
    print("Points created")

    for batch_points in create_batch_points(points, batch_size=batch_size):
        try:
            client.upsert(
                collection_name="quantized_collection_for_code_search",
                points=batch_points,
            )
            client.upsert(
                collection_name="normal_collection_for_code_search",
                points=batch_points,
            )
        except Exception as e:
            print(f"Error during upsert: {e}")

### Inference / Retrieval

Let's check retrieval for a sample query

In [5]:
query = "Scan a binary tree in level order"
query_embedding = list(text_model.embed(query))
query_code_embedding = list(code_model.embed(query))

In [6]:
import time
from qdrant_client import QdrantClient, models

client = QdrantClient("localhost:6333")
# Time profile for quantized search
start_time = time.time()
quantized_results = client.search(
    collection_name="quantized_collection_for_code_search",
    query_vector={"name":"text","vector":query_embedding[0]},
    search_params=models.SearchParams(
        quantization=models.QuantizationSearchParams(rescore=True)
    ),
    limit=10,
)
quantized_search_time = time.time() - start_time
print(f"Quantized search time: {quantized_search_time} seconds")

# Time profile for normal search
start_time = time.time()
normal_results = client.search(
    collection_name="normal_collection_for_code_search",
    query_vector={"name":"text","vector":query_embedding[0]},
    limit=10,
)
normal_search_time = time.time() - start_time
print(f"Normal search time: {normal_search_time} seconds")


Quantized search time: 0.21096587181091309 seconds
Normal search time: 0.10556602478027344 seconds


The Quantized Approach takes way less time speeding up the time by almost 3x times 

In [64]:
import time
from qdrant_client import QdrantClient, models

client = QdrantClient("localhost:6333")
# Time profile for quantized search
start_time = time.time()
quantized_results = client.search(
    collection_name="quantized_collection_for_code_search",
    query_vector={"name":"code","vector":query_code_embedding[0]},
    search_params=models.SearchParams(
        quantization=models.QuantizationSearchParams(rescore=True),
        hnsw_ef=128
    ),
    limit=10,
)
quantized_search_time = time.time() - start_time
print(f"Quantized search time: {quantized_search_time} seconds")

# Time profile for normal search
start_time = time.time()
normal_results = client.search(
    collection_name="normal_collection_for_code_search",
    query_vector={"name":"code","vector":query_code_embedding[0]},
    search_params=models.SearchParams(
        hnsw_ef=128
    ),
    limit=10,
)
normal_search_time = time.time() - start_time
print(f"Normal search time: {normal_search_time} seconds")


Quantized search time: 0.04339194297790527 seconds
Normal search time: 0.5557098388671875 seconds


Here for the same query , searching over code field through quantized approach is 12x times faster!

In [65]:
# Benchmark search time for different queries and present in a tabular format
from tabulate import tabulate

queries = [
    "sort an array using quicksort",
    "implement binary search in a sorted array",
    "reverse a linked list",
    "find the depth of a binary tree",
    "merge two sorted linked lists",
    "detect a cycle in a graph",
    "find the median of two sorted arrays",
    "implement a stack using queues",
    "find the kth largest element in an array",
    "check if a string is a valid palindrome"
]

results_table = []

for query in queries:
    query_embedding = list(text_model.embed(query))
    query_code_embedding = list(code_model.embed(query))
    
    # Time profile for quantized search
    start_time = time.time()
    quantized_results = client.search(
        collection_name="quantized_collection_for_code_search",
        query_vector={"name":"code","vector":query_code_embedding[0]},
        search_params=models.SearchParams(
            quantization=models.QuantizationSearchParams(rescore=False),
            hnsw_ef=128
        ),
        score_threshold=0.5,
        limit=100,

    )
    quantized_search_time = time.time() - start_time

    # Time profile for normal search
    start_time = time.time()
    normal_results = client.search(
        collection_name="normal_collection_for_code_search",
        query_vector={"name":"code","vector":query_code_embedding[0]},
        search_params=models.SearchParams(
            hnsw_ef=128
        ),
        score_threshold=0.5,
        limit=100,
    )
    normal_search_time = time.time() - start_time


    # Time profile for quantized search over text fields
    start_time = time.time()
    quantized_results_text = client.search(
        collection_name="quantized_collection_for_code_search",
        query_vector={"name":"text","vector":query_embedding[0]},
        search_params=models.SearchParams(
            quantization=models.QuantizationSearchParams(rescore=False),
            hnsw_ef=128
        ),
        score_threshold=0.5,
        limit=100,
    )
    quantized_search_time_text = time.time() - start_time

    # Time profile for normal search over text fields
    start_time = time.time()
    normal_results_text = client.search(
        collection_name="normal_collection_for_code_search",
        query_vector={"name":"text","vector":query_embedding[0]},
        search_params=models.SearchParams(
            hnsw_ef=128
        ),
        score_threshold=0.5,
        limit=100,
    )
    normal_search_time_text = time.time() - start_time
    # Calculate difference and scale
    time_difference = normal_search_time - quantized_search_time
    scale = normal_search_time / quantized_search_time if quantized_search_time != 0 else float('inf')
    time_difference_text = normal_search_time_text - quantized_search_time_text
    scale_text = normal_search_time_text / quantized_search_time_text if quantized_search_time_text != 0 else float('inf')
    # Append results to the table
    results_table.append([query, quantized_search_time, normal_search_time, time_difference, scale, quantized_search_time_text, normal_search_time_text, time_difference_text, scale_text])

# Print the results in a tabular format
headers = ["Query", "Quantized Search Time (s)", "Normal Search Time (s)", "Difference (s)", "Scale", "Quantized Text Search Time (s)", "Normal Text Search Time (s)", "Difference (s)", "Scale"]
print(tabulate(results_table, headers=headers, tablefmt="grid"))


+-------------------------------------------+-----------------------------+--------------------------+------------------+-----------+----------------------------------+-------------------------------+------------------+---------+
| Query                                     |   Quantized Search Time (s) |   Normal Search Time (s) |   Difference (s) |     Scale |   Quantized Text Search Time (s) |   Normal Text Search Time (s) |   Difference (s) |   Scale |
+===========================================+=============================+==========================+==================+===========+==================================+===============================+==================+=========+
| sort an array using quicksort             |                   0.257229  |                1.26377   |       1.00654    |  4.91303  |                       0.0795069  |                     0.40544   |       0.325933   | 5.09943 |
+-------------------------------------------+-----------------------------+-----

In [98]:
results_table

[['sort an array using quicksort',
  0.2572290897369385,
  1.263772964477539,
  1.0065438747406006,
  4.913025061706539,
  0.07950687408447266,
  0.4054398536682129,
  0.32593297958374023,
  5.099431443342249],
 ['implement binary search in a sorted array',
  0.08635592460632324,
  0.6522817611694336,
  0.5659258365631104,
  7.553410656455082,
  0.06979823112487793,
  0.21520590782165527,
  0.14540767669677734,
  3.0832573312155214],
 ['reverse a linked list',
  0.060385704040527344,
  0.5740752220153809,
  0.5136895179748535,
  9.506806803645036,
  0.04556918144226074,
  0.3635702133178711,
  0.31800103187561035,
  7.9784231757276425],
 ['find the depth of a binary tree',
  0.05312085151672363,
  0.5973610877990723,
  0.5442402362823486,
  11.24532214268082,
  0.033447980880737305,
  0.23994016647338867,
  0.20649218559265137,
  7.1735321581569735],
 ['merge two sorted linked lists',
  0.04495501518249512,
  0.8352138996124268,
  0.7902588844299316,
  18.578881493463445,
  0.026173353

In [103]:
import numpy as np
# Calculate mean search times for quantized and normal search times
mean_quantized_search_time = np.mean([row[1] for row in results_table[1:]])
mean_normal_search_time = np.mean([row[2] for row in results_table[1:]])
mean_quantized_search_time_text = np.mean([row[5] for row in results_table[1:]])
mean_normal_search_time_text = np.mean([row[6] for row in results_table[1:]])

print(f"Mean Quantized Code Search Time (s): {mean_quantized_search_time}")
print(f"Mean Normal Code Search Time (s): {mean_normal_search_time}")
print(f"Scale: { mean_normal_search_time / mean_quantized_search_time}")
print("--------------------------------")
print(f"Mean Quantized Text Search Time (s): {mean_quantized_search_time_text}")
print(f"Mean Normal Text Search Time (s): {mean_normal_search_time_text}")
print(f"Scale: {mean_normal_search_time_text / mean_quantized_search_time_text}")


Mean Quantized Code Search Time (s): 0.04558769861857096
Mean Normal Code Search Time (s): 0.5089571211073134
Scale: 11.164352150472027
--------------------------------
Mean Quantized Text Search Time (s): 0.029638740751478408
Mean Normal Text Search Time (s): 0.1786990695529514
Scale: 6.0292396040488905


In [66]:
from tabulate import tabulate

# Compare ID and score differences between quantized and normal results
def compare_results(quantized_results, normal_results):
    differences = []
    for q_result, n_result in zip(quantized_results, normal_results):
        if q_result.id == n_result.id:
            differences.append([
                q_result.id,
                n_result.id,
                q_result.score,
                n_result.score
            ])
    return differences

# Call the function and print differences in a tabular format
differences = compare_results(quantized_results, normal_results)
headers = ["Quantized ID", "Normal ID", "Quantized Score", "Normal Score"]
print(tabulate(differences, headers=headers, tablefmt="grid"))

+----------------+-------------+-------------------+----------------+
|   Quantized ID |   Normal ID |   Quantized Score |   Normal Score |
+================+=============+===================+================+
|          98250 |       98250 |          0.780283 |       0.775797 |
+----------------+-------------+-------------------+----------------+
|          86608 |       86608 |          0.779882 |       0.774726 |
+----------------+-------------+-------------------+----------------+
|         143581 |      143581 |          0.773003 |       0.771941 |
+----------------+-------------+-------------------+----------------+
|          29766 |       29766 |          0.764984 |       0.762938 |
+----------------+-------------+-------------------+----------------+
|         178999 |      178999 |          0.759316 |       0.757656 |
+----------------+-------------+-------------------+----------------+
|          12739 |       12739 |          0.758219 |       0.756247 |
+----------------+--

In [73]:
# Hybrid Search over text and code fields
query = "Scan a binary tree in level order"
query_embedding = list(text_model.embed(query))
query_code_embedding = list(code_model.embed(query))

# text based results
text_results = client.search(
    collection_name="quantized_collection_for_code_search",
    query_vector={"name":"text","vector":query_embedding[0]},
    limit=10,
)

# code based results
code_results = client.search(
    collection_name="quantized_collection_for_code_search",
    query_vector={"name":"code","vector":query_code_embedding[0]},
    limit=10,
)

# hybrid search results
hybrid_search_results = client.query_points(
    collection_name="quantized_collection_for_code_search",
    prefetch=[
                models.Prefetch(
                    query=query_embedding[0],
                    using="text",
                    limit=10,
                ),
                models.Prefetch(
                    query=query_code_embedding[0],
                    using="code",
                    limit=10,
                ),
            ],
    query=models.FusionQuery(fusion=models.Fusion.RRF),
)


In [75]:
text_results

[ScoredPoint(id=124705, version=487, score=0.6409397, payload={'file_content': "Problem Description: Let *T* be arbitrary binary tree — tree, every vertex of which has no more than two children. Given tree is rooted, so there exists only one vertex which doesn't have a parent — it's the root of a tree. Every vertex has an integer number written on it. Following algorithm is run on every value from the tree *T*:\n 1.  Set pointer to the root of a tree. 1.  Return success if the value in the current vertex is equal to the number you are looking for 1.  Go to the left child of the vertex if the value in the current vertex is greater than the number you are looking for 1.  Go to the right child of the vertex if the value in the current vertex is less than the number you are looking for 1.  Return fail if you try to go to the vertex that doesn't exist \nHere is the pseudo-code of the described algorithm: \n\nThe described algorithm works correctly if the tree is binary search tree (i.e. for

In [74]:
code_results

[ScoredPoint(id=3959, version=15, score=0.62690353, payload={'file_content': "Problem Description: Jon fought bravely to rescue the wildlings who were attacked by the white-walkers at Hardhome. On his arrival, Sam tells him that he wants to go to Oldtown to train at the Citadel to become a maester, so he can return and take the deceased Aemon's place as maester of Castle Black. Jon agrees to Sam's proposal and Sam sets off his journey to the Citadel. However becoming a trainee at the Citadel is not a cakewalk and hence the maesters at the Citadel gave Sam a problem to test his eligibility. \n\nInitially Sam has a list with a single element *n*. Then he has to perform certain operations on this list. In each operation Sam must remove any element *x*, such that *x*<=&gt;<=1, from the list and insert at the same position , ,  sequentially. He must continue with these operations until all the elements in the list are either 0 or 1.\n\nNow the masters want the total number of 1s in the rang

## Reciprocal Rank Fusion
Reciprocal Rank Fusion (RRF) is a technique used to combine multiple rankings into a single ranking. It is a simple and effective way to improve the quality of the ranking.

RRF works by assigning a score to each document based on its rank in each individual ranking. The score for a document is calculated as the sum of the reciprocal of its rank plus a constant k. The formula for the RRF score is:
 
$$\text{RRF score} = \sum_{i=1}^{n} \frac{1}{k + r_i}$$

where \( n \) is the number of rankings, \( r_i \) is the rank of the document in the \( i \)-th ranking, and \( k \) is a constant (typically set to 60) to ensure that all documents receive a non-zero score.

RRF does not require the rankings to be normalized or the scores to be on the same scale, making it a robust and flexible method for combining rankings from different sources.

In [85]:
def calculate_rrf(rankings, k=1):
    scores = {}
    # Initialize scores for each document
    for ranker in rankings:
        for document in rankings[ranker]:
            scores[document.id] = 0
    
    # Calculate RRF scores
    for ranker in rankings:
        for i, document in enumerate(rankings[ranker]):
            scores[document.id] += 1 / (k + i + 1)
    
    return scores

def get_final_ranking(scores):
    # Sort documents by scores in descending order
    return sorted(scores.items(), key=lambda item: item[1], reverse=True)

# Prepare rankings from text and code results
rankings = {
    "text": text_results,
    "code": code_results
}

# Calculate RRF scores and get final ranking
rrf_scores = calculate_rrf(rankings)
custom_hybrid_search_results = get_final_ranking(rrf_scores)
custom_hybrid_search_results[:10]

[(124705, 0.5),
 (3959, 0.5),
 (50331, 0.3333333333333333),
 (248884, 0.3333333333333333),
 (91121, 0.25),
 (211046, 0.25),
 (60346, 0.2),
 (120520, 0.2),
 (152431, 0.16666666666666666),
 (122374, 0.16666666666666666)]

In [2]:
hybrid_search_results.points

NameError: name 'hybrid_search_results' is not defined

In [ ]:
import openai
from qdrant_client import QdrantClient

client = QdrantClient("localhost:6333")


def fetch_results_from_qdrant(query, type, number_of_docs):
    code_vector = list(code_model.embed(query))
    text_vector = list(text_model.embed(query))
    if type=="text":
        search_results = client.search(
            collection_name="quantized_collection_for_code_search",
            query_vector={"name":"text","vector":text_vector[0]},
            limit=number_of_docs,
            search_type="hybrid"
        )
        return [result.payload.get("file_content", "")+"\n\n"+result.payload.get("code", "") for result in search_results]
    elif type=="code":
        search_results = client.search(
            collection_name="quantized_collection_for_code_search",
            query_vector={"name":"code","vector":code_vector[0]},
            limit=number_of_docs,
            search_type="hybrid"
        )
        return [result.payload.get("file_content", "")+"\n\n"+result.payload.get("code", "") for result in search_results]
    elif type=="hybrid":
        search_results = client.query_points(
            collection_name="quantized_collection_for_code_search",
            prefetch=[
                models.Prefetch(query=text_vector[0], using="text", limit=number_of_docs),
                models.Prefetch(query=code_vector[0], using="code", limit=number_of_docs),
            ],
            query=models.FusionQuery(fusion=models.Fusion.RRF),
        )
        return [result.payload.get("file_content", "")+"\n\n"+result.payload.get("code", "") for result in search_results.points]

def openai_call(query, type="hybrid", number_of_docs=10):
    # Fetch results from Qdrant
    search_results = fetch_results_from_qdrant(query, type, number_of_docs)
    
    # Extract file content and code from the payload
    docs = []
    for result in search_results:
        file_content = result.payload.get("file_content", "")
        code = result.payload.get("code", "")
        docs.append({"file_content": file_content, "code": code})
    
    # Prepare the answer
    answer = {
        "query": query,
        "type": type,
        "number_of_docs": number_of_docs,
        "docs_fetched": docs
    }
    
    return answer

# Example usage
query = "example query"
result = openai_call(query, type="hybrid", number_of_docs=10)
print(result)
